In [1]:
# PRÉ-TRATAMENTO - VERSÃO FINAL OTIMIZADA

import pandas as pd
import numpy as np
import os
import gc
import time
import psutil
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
from pathlib import Path

# Configurações
ENTRADA = "../../banco/parquet_unificado/sih_rs.parquet"
SAIDA = "../../banco/parquet_unificado/sih_rs_tratado.parquet"
BACKUP_DIR = "../../banco/backups"
LIMITE_MB = 500
BATCH_SIZE = 500_000

print("PRÉ-TRATAMENTO DE DADOS - VERSÃO FINAL")
print("=" * 50)

def verificar_memoria():
   memory = psutil.virtual_memory()
   if memory.percent > 85:
       gc.collect()
       time.sleep(1)
   return memory.percent

def tratar_num_filhos(df):
   if 'NUM_FILHOS' in df.columns:
       df['NUM_FILHOS'] = pd.to_numeric(df['NUM_FILHOS'], errors='coerce').fillna(0).astype(int)
   return df

def tratar_instrucao(df):
   if 'INSTRU' not in df.columns:
       return df
   df['INSTRU'] = df['INSTRU'].astype(str).str.zfill(2)
   df['INSTRU'] = df['INSTRU'].replace(['00', 'nan'], '0')
   return df

def padronizar_cids(df, colunas_cid):
   colunas_existentes = [col for col in colunas_cid if col in df.columns]
   if not colunas_existentes:
       return df
   
   for col in colunas_existentes:
       df[col] = df[col].astype(str).str.upper().str.strip()
       mask = df[col].isin(['0000', 'NAN', ''])
       df.loc[mask, col] = np.nan
   return df

def tratar_cids(df):
   colunas_cid = [
       'DIAG_PRINC', 'DIAG_SECUN', 'CID_NOTIF', 'CID_ASSO', 'CID_MORTE'
   ] + [f'DIAGSEC{i}' for i in range(1, 10)]
   return padronizar_cids(df, colunas_cid)

def tratar_idade(df):
   if 'IDADE' in df.columns:
       df['IDADE'] = pd.to_numeric(df['IDADE'], errors='coerce').fillna(0).astype(float)

   if 'COD_IDADE' in df.columns:
       df['COD_IDADE'] = pd.to_numeric(df['COD_IDADE'], errors='coerce').fillna(0).astype(int)
       
       df.loc[df['COD_IDADE'] == 1, 'IDADE'] = 0  
       df.loc[df['COD_IDADE'] == 2, 'IDADE'] = (df.loc[df['COD_IDADE'] == 2, 'IDADE'] / 365).round(1) 
       df.loc[df['COD_IDADE'] == 3, 'IDADE'] = (df.loc[df['COD_IDADE'] == 3, 'IDADE'] / 12).round(1)   
       
       df = df.drop('COD_IDADE', axis=1)
       
   return df

def tratar_sexo(df):
   if 'SEXO' not in df.columns:
       return df
   df['SEXO'] = pd.to_numeric(df['SEXO'], errors='coerce').astype('Int64')
   return df

def tratar_datas(df):
   colunas_datas = ['DT_INTER', 'DT_SAIDA', 'NASC']
   colunas_existentes = [col for col in colunas_datas if col in df.columns]
   
   for col in colunas_existentes:
       df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y%m%d')
   
   return df

def tratar_valores(df):
   colunas_valores = ['VAL_SH', 'VAL_SP', 'VAL_TOT', 'VAL_UTI']
   
   for col in colunas_valores:
       if col in df.columns:
           df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
   return df

def tratar_inteiros(df):
   colunas_inteiras = ['UTI_MES_TO', 'UTI_INT_TO', 'DIAR_ACOM', 'QT_DIARIAS', 'DIAS_PERM']
   
   for col in colunas_inteiras:
       if col in df.columns:
           df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
   return df

def contrair_naih(df):
   print("Contraindo dados por N_AIH...")
   registros_antes = len(df)
   aihs_unicas_antes = df['N_AIH'].nunique()
   
   if registros_antes == aihs_unicas_antes:
       print("Não há duplicatas por N_AIH.")
       return df
   
   colunas_soma = ['VAL_SH', 'VAL_SP', 'VAL_TOT', 'VAL_UTI', 'QT_DIARIAS', 'DIAS_PERM']
   colunas_media = ['UTI_MES_TO', 'UTI_INT_TO', 'DIAR_ACOM', 'IDADE']
   
   agg_dict = {}
   
   for col in df.columns:
       if col == 'N_AIH':
           continue
       elif col in colunas_soma:
           agg_dict[col] = 'sum'
       elif col in colunas_media:
           agg_dict[col] = 'mean'
       else:
           agg_dict[col] = 'first'
   
   df_contraido = df.groupby('N_AIH').agg(agg_dict).reset_index()
   
   for col in colunas_media:
       if col in df_contraido.columns:
           df_contraido[col] = df_contraido[col].round(1)
   
   print(f"Registros: {registros_antes:,} → {len(df_contraido):,} ({((registros_antes - len(df_contraido)) / registros_antes * 100):.1f}% redução)")
   
   return df_contraido

def aplicar_tratamentos(df, tratamentos):
   for func in tratamentos:
       df = func(df)
   return df

def processar_chunks(parquet_entrada, parquet_saida, tratamentos, batch_size=500_000):
   print(f"Processando em chunks de {batch_size:,} registros...")
   
   parquet_file = pq.ParquetFile(parquet_entrada)
   chunks_processados = []
   
   batch_num = 0
   for batch in parquet_file.iter_batches(batch_size=batch_size):
       batch_num += 1
       df_chunk = batch.to_pandas()
       
       if batch_num % 10 == 0:
           print(f"Processando chunk {batch_num}...")
       
       tratamentos_chunk = [t for t in tratamentos if t.__name__ != 'contrair_naih']
       df_chunk = aplicar_tratamentos(df_chunk, tratamentos_chunk)
       chunks_processados.append(df_chunk)
       
   print("Concatenando chunks...")
   df_completo = pd.concat(chunks_processados, ignore_index=True)
   
   if any(t.__name__ == 'contrair_naih' for t in tratamentos):
       df_completo = contrair_naih(df_completo)

   print("Salvando arquivo...")
   table = pa.Table.from_pandas(df_completo, preserve_index=False)
   pq.write_table(table, parquet_saida, compression="snappy")
   
   return len(df_completo)

def processar_simples(parquet_entrada, parquet_saida, tratamentos):
   print("Processando arquivo na memória...")
   
   df = pd.read_parquet(parquet_entrada)
   print(f"Carregado: {len(df):,} registros")
   
   for tratamento in tratamentos:
       df = tratamento(df)
   
   print("Salvando arquivo...")
   df.to_parquet(parquet_saida, index=False, engine="pyarrow", compression="snappy")
   
   return len(df)

def processar_tratamento(parquet_entrada, parquet_saida, limite_mb=500, batch_size=500_000):
   tratamentos = [
       tratar_num_filhos,
       tratar_instrucao,
       tratar_cids,
       tratar_idade,
       tratar_sexo,
       tratar_datas,
       tratar_valores,
       tratar_inteiros,
       contrair_naih
   ]
   
   if not os.path.exists(parquet_entrada):
       raise FileNotFoundError(f"Arquivo {parquet_entrada} não encontrado!")
   
   tamanho_mb = os.path.getsize(parquet_entrada) / (1024 * 1024)
   print(f"Arquivo entrada: {tamanho_mb:.1f} MB")
   
   Path(parquet_saida).parent.mkdir(parents=True, exist_ok=True)
   
   if os.path.exists(parquet_saida):
       timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
       backup_path = f"{BACKUP_DIR}/sih_rs_tratado_backup_{timestamp}.parquet"
       Path(BACKUP_DIR).mkdir(parents=True, exist_ok=True)
       os.rename(parquet_saida, backup_path)
       print(f"Backup criado: {backup_path}")
   
   inicio = time.time()
   
   if tamanho_mb > limite_mb:
       print("Usando processamento em chunks...")
       registros_finais = processar_chunks(parquet_entrada, parquet_saida, tratamentos, batch_size)
   else:
       registros_finais = processar_simples(parquet_entrada, parquet_saida, tratamentos)
   
   tempo_total = time.time() - inicio
   tamanho_final_mb = os.path.getsize(parquet_saida) / (1024 * 1024)
   
   print("\n" + "="*50)
   print("TRATAMENTO CONCLUÍDO!")
   print("="*50)
   print(f"Registros finais: {registros_finais:,}")
   print(f"Tamanho final: {tamanho_final_mb:.1f} MB")
   print(f"Tempo: {tempo_total:.1f}s ({tempo_total/60:.1f} min)")
   print(f"Próximo: executar bd.ipynb")
   
   return registros_finais

# Verificar arquivo de entrada
if not os.path.exists(ENTRADA):
   raise FileNotFoundError(f"Arquivo não encontrado: {ENTRADA}")

print(f"Entrada: {ENTRADA}")
print(f"Saída: {SAIDA}")

# Executar processamento
try:
   resultado = processar_tratamento(ENTRADA, SAIDA, LIMITE_MB, BATCH_SIZE)
   print(f"\nSUCESSO! {resultado:,} registros processados.")
   
except Exception as e:
   print(f"\nERRO: {e}")
   raise

finally:
   gc.collect()
   print(f"Finalizado em {datetime.now()}")

PRÉ-TRATAMENTO DE DADOS - VERSÃO FINAL
Entrada: ../../banco/parquet_unificado/sih_rs.parquet
Saída: ../../banco/parquet_unificado/sih_rs_tratado.parquet
Arquivo entrada: 792.4 MB
Usando processamento em chunks...
Processando em chunks de 500,000 registros...
Processando chunk 10...
Processando chunk 20...
Processando chunk 30...
Processando chunk 40...
Concatenando chunks...
Contraindo dados por N_AIH...
Registros: 22,183,490 → 11,022,199 (50.3% redução)
Salvando arquivo...

TRATAMENTO CONCLUÍDO!
Registros finais: 11,022,199
Tamanho final: 353.6 MB
Tempo: 362.3s (6.0 min)
Próximo: executar bd.ipynb

SUCESSO! 11,022,199 registros processados.
Finalizado em 2025-06-06 17:12:49.626376
